We have used the "US Treasury Zero-Coupon Yield curve" for 30 years and "3-month T-bill yield". Data can be downloaded from FRED.

In [1]:
import pandas as pd
import glob
from functools import reduce
import numpy as np
from scipy.interpolate import CubicSpline

In [2]:
# Path to your CSV files
path = 'D:/python/MF728project/*.csv'  # Adjust this to the path where your CSV files are stored
all_files = glob.glob(path)
all_files

['D:/python/MF728project\\three_month_tbills.csv',
 'D:/python/MF728project\\zero_coupon_yields_1.csv',
 'D:/python/MF728project\\zero_coupon_yields_10.csv',
 'D:/python/MF728project\\zero_coupon_yields_2.csv',
 'D:/python/MF728project\\zero_coupon_yields_20.csv',
 'D:/python/MF728project\\zero_coupon_yields_3.csv',
 'D:/python/MF728project\\zero_coupon_yields_30.csv',
 'D:/python/MF728project\\zero_coupon_yields_5.csv',
 'D:/python/MF728project\\zero_coupon_yields_7.csv']

In [3]:
# List to hold data from each file
dataframes = []

# Read each CSV file and append to the list
for filename in all_files:
    df = pd.read_csv(filename)
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
    dataframes.append(df)

# Merge all dataframes on the 'DATE' column
merged_df = reduce(lambda left, right: pd.merge(left, right, on='DATE', how='outer'), dataframes)

# Filter rows by date range
start_date = '1981-09-01'
end_date = '2024-04-15'
merged_df = merged_df[(merged_df['DATE'] >= start_date) & (merged_df['DATE'] <= end_date)]

In [4]:
merged_df.head()

,DATE,DGS3MO,DGS1,DGS10,DGS2,DGS20,DGS3,DGS30,DGS5,DGS7
0,1981-09-01,17.01,17.060,15.41,16.780,15.09,16.57,14.70,16.12,15.71
1,1981-09-02,16.65,17.160,15.40,16.780,15.11,16.43,14.70,16.09,15.75
2,1981-09-03,16.96,17.310,15.48,16.900,15.21,16.48,14.82,16.14,15.78
3,1981-09-04,16.64,17.240,15.51,16.890,15.24,16.52,14.84,16.17,15.86
4,1981-09-07,.,.,.,.,.,.,.,.,.


In [5]:
# Make sure that 'DATE' is the DataFrame index for easier manipulation
merged_df.set_index('DATE', inplace=True)

# Convert all columns to numeric as they might be read as strings (especially if there are '.' from Excel)
merged_df = merged_df.apply(pd.to_numeric, errors='coerce')

# Now use interpolate with a time method, which will do a time-weighted interpolation
merged_df.interpolate(method='time', inplace=True)

merged_df.head()

,DGS3MO,DGS1,DGS10,DGS2,DGS20,DGS3,DGS30,DGS5,DGS7
DATE,,,,,,,,,
1981-09-01,17.010,17.0600,15.41,16.780,15.09,16.57,14.7000,16.120,15.710
1981-09-02,16.650,17.1600,15.40,16.780,15.11,16.43,14.7000,16.090,15.750
1981-09-03,16.960,17.3100,15.48,16.900,15.21,16.48,14.8200,16.140,15.780
1981-09-04,16.640,17.2400,15.51,16.890,15.24,16.52,14.8400,16.170,15.860
1981-09-07,16.565,17.2775,15.57,16.935,15.33,16.55,14.9525,16.215,15.905


In [6]:
merged_df[['DGS3MO']].to_csv('D:/python/MF728project/DGS3MO.csv', header=True)
merged_df.drop(columns=['DGS3MO']).to_csv('D:/python/MF728project/DGS1_7.csv', header=True)

In [24]:
merged_df.describe()

,DGS3MO,DGS1,DGS10,DGS2,DGS20,DGS3,DGS30,DGS5,DGS7
count,11120.000000,11120.000000,11120.000000,11120.000000,11120.000000,11120.000000,11120.000000,11120.000000,11120.000000
mean,3.803417,4.136341,5.422832,4.448086,5.609749,4.639107,5.849459,4.969471,5.235061
std,3.194697,3.356785,3.124419,3.433397,2.824321,3.401959,2.872706,3.305004,3.225921
min,0.000000,0.040000,0.520000,0.090000,0.870000,0.100000,0.990000,0.190000,0.360000
25%,0.810000,1.120000,2.870000,1.330000,3.400000,1.557500,3.570000,2.110000,2.550000
50%,3.750000,4.110000,4.760000,4.271250,5.310000,4.410000,5.240000,4.550000,4.690000
75%,5.700000,6.070000,7.290000,6.380000,6.831349,6.550000,7.580000,6.810000,7.080000
max,17.010000,17.310000,15.840000,16.950000,15.780000,16.590000,15.210000,16.270000,16.050000


In [26]:
merged_df.isnull().sum()

DGS3MO    0
DGS1      0
DGS10     0
DGS2      0
DGS20     0
DGS3      0
DGS30     0
DGS5      0
DGS7      0
dtype: int64

In [30]:
# Initialize a dictionary to hold the interpolated yield curves
interpolated_curves = {}

# Known maturities
known_maturities = np.array([1, 2, 3, 5, 7, 10, 20, 30])

# Interpolating for each date
for date, row in merged_df.iterrows():
    # Extract the yields for the known maturities
    known_yields = row[['DGS1', 'DGS2', 'DGS3', 'DGS5', 'DGS7', 'DGS10', 'DGS20', 'DGS30']].values
    # Setup cubic spline interpolation
    cs = CubicSpline(known_maturities, known_yields)
    # Maturities to interpolate
    full_range_maturities = np.arange(1, 31)
    # Interpolate yields for the full range
    interpolated_yields = cs(full_range_maturities)
    # Store the interpolated yields in the dictionary
    interpolated_curves[date] = interpolated_yields

# Convert the dictionary to a DataFrame
interpolated_df = pd.DataFrame(interpolated_curves).T
interpolated_df.columns = [f'DGS{m}' for m in full_range_maturities]  # Name the columns according to maturity

In [31]:
interpolated_df['DGS3MO'] = merged_df['DGS3MO']
interpolated_df.head()

,DGS1,DGS2,DGS3,DGS4,DGS5,DGS6,DGS7,DGS8,DGS9,DGS10,...,DGS22,DGS23,DGS24,DGS25,DGS26,DGS27,DGS28,DGS29,DGS30,DGS3MO
1981-09-01,17.0600,16.780,16.57,16.350396,16.120,15.898415,15.710,15.573295,15.478799,15.41,...,15.049236,15.025332,14.997763,14.965539,14.927672,14.883173,14.831054,14.770326,14.7000,17.010
1981-09-02,17.1600,16.780,16.43,16.225968,16.090,15.924878,15.750,15.604572,15.490415,15.40,...,15.093330,15.078842,15.057928,15.028848,14.989861,14.939224,14.875196,14.796035,14.7000,16.650
1981-09-03,17.3100,16.900,16.48,16.262789,16.140,15.967593,15.780,15.640102,15.546094,15.48,...,15.177243,15.156281,15.130776,15.099616,15.061687,15.015875,14.961065,14.896145,14.8200,16.960
1981-09-04,17.2400,16.890,16.52,16.302912,16.170,16.019604,15.860,15.720523,15.604793,15.51,...,15.232570,15.222048,15.204398,15.177676,15.139941,15.089248,15.023655,14.941220,14.8400,16.640
1981-09-07,17.2775,16.935,16.55,16.336354,16.215,16.066772,15.905,15.767511,15.657616,15.57,...,15.324179,15.314622,15.298188,15.273045,15.237358,15.189294,15.127019,15.048698,14.9525,16.565
